# Hello ~~world~~ Phi-3!

## Prerequisites 

### Importing Required Libraries
In this cell, we are importing the necessary libraries. `MLClient` is the main class that we use to interact with Azure AI. `DefaultAzureCredential` and `InteractiveBrowserCredential` are used for authentication purposes. The `os` library is used to access environment variables.

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import os
```

### Setting Up Credentials
Here, we are setting up the credentials to authenticate with Azure. We first try to use the `DefaultAzureCredential`. If that fails (for example, if we are running the code on a machine that is not logged into Azure), we fall back to using `InteractiveBrowserCredential`, which will prompt the user to log in.

```python
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
```

### Creating an MLClient for Workspace
In this cell, we create an `MLClient` for our Azure AI workspace. We use environment variables to get the subscription ID, resource group name, and workspace name.

```python
workspace_ml_client = MLClient(
    credential,
    subscription_id=os.getenv("SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("RESOURCE_GROUP"),
    workspace_name=os.getenv("WORKSPACE_NAME"),
)
```

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import os

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id=os.getenv("SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("RESOURCE_GROUP"),
    workspace_name=os.getenv("WORKSPACE_NAME"),
)

### Optional: Loading and Preparing the Dataset
This step is optional and not required to demo Phi-3. However, if you want to experiment with different topics for the model, you can use a dataset. In this case, we are using the `ultrachat_200k` dataset from Hugging Face. First, we import the necessary libraries: `pandas` for data manipulation and `datasets` for loading the dataset.
```python
import pandas as pd
from datasets import load_dataset
```
Next, we load the [ultrachat_200k dataset from Hugging Face](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k) and select the `test_sft` split.

```python
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")["test_sft"]
```
We then convert the dataset into a pandas DataFrame and drop the 'prompt_id' and 'messages' columns. These columns are not needed for our current task.
```python
df = pd.DataFrame(dataset).drop(columns=["prompt_id", "messages"])
```
Finally, we display a random sample of 5 rows from the DataFrame. This gives us a quick look at the data we'll be working with.
```python
df.sample(5)
```

In [ ]:
# Import necessary libraries
import pandas as pd
from datasets import load_dataset

# Load the 'ultrachat_200k' dataset from HuggingFace and select the 'test_sft' split
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")["test_sft"]

# Convert the dataset into a pandas DataFrame and drop the 'prompt_id' and 'messages' columns
df = pd.DataFrame(dataset).drop(columns=["prompt_id", "messages"])

# Display a random sample of 5 rows from the DataFrame
df.sample(5)

### Optional (continued): Selecting a Random Sample
In this section, we are selecting a random sample from our dataset to use as a test case for Phi-3. First, we sample 5 random examples from the DataFrame and convert them to a list. This gives us a small set of examples to choose from.
```python
examples = df.sample(5).values.tolist()
```
Next, we convert the examples to a JSON string. This is done for pretty printing, which makes the examples easier to read.
```python
examples_json = json.dumps(examples, indent=2)
```
We then select a random index from the examples. This is done using the random.randint function, which returns a random integer within the specified range.
```python
i = random.randint(0, len(examples) - 1)
```
We use this random index to select an example from our list.
```python
sample = examples[i]
print(sample)
```
This process ensures that we have a diverse range of topics to test our model with, and that the testing process is as unbiased as possible.

In [ ]:
import random
import json

# Sample 5 random examples from the DataFrame and convert to list
examples = df.sample(5).values.tolist()

# Convert the examples to a JSON string for pretty printing
examples_json = json.dumps(examples, indent=2)

# Select a random index from the examples
i = random.randint(0, len(examples) - 1)

# Get the example at the random index
sample = examples[i]

# Print the selected example
print(sample)

### Invoking the Phi-3 Model
In this section, we are invoking the Phi-3 model to generate a response to a user's question.

First, we define the input data. This includes the user's message and some parameters for the model. The parameters control the randomness of the model's output.

```python
messages = {
    "input_data": { "input_string": [ { "role": "user", "content": "I am going to Paris, what should I see?" } ],
        "parameters": { "temperature": 0.7, "top_p": 0.9, "do_sample": True, "max_new_tokens": 1000 } }
}
```
Next, we write the input data to a temporary file. This is necessary because the `invoke` method of the `workspace_ml_client.online_endpoints` object requires a file as input.
```python
with tempfile.NamedTemporaryFile(suffix=".json", delete=False, mode='w') as temp:
    json.dump(messages, temp)
    temp_file_name = temp.name
```
We then invoke the Phi-3 model and get the response. The `invoke` method sends the input data to the model and returns the model's output.
```python
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name="aistudio-nbrady-phi-3",
    deployment_name="phi-3-mini-4k-instruct",
    request_file=temp_file_name,
)
```
After getting the response, we parse it and add it to the input data. This allows us to keep track of the conversation history. Finally, we print the updated input data. This includes the user's message and the model's response.
```python
response_json = json.loads(response)["output"]
response_dict = {'content': response_json, 'role': 'assistant'}
messages['input_data']['input_string'].append(response_dict)
print(json.dumps(messages["input_data"]["input_string"],indent=2))
```
This process allows us to interact with the Phi-3 model and get its responses to various inputs.

In [ ]:
import json
import tempfile

# Define the input data
messages = { "input_data": { 
    "input_string": [ { "role": "user", "content": "I am going to Paris, what should I see?"} ], 
    "parameters": { "temperature": 0.7, "top_p": 0.9, "do_sample": True, "max_new_tokens": 1000, }, } 
}

# Write the input data to a temporary file
with tempfile.NamedTemporaryFile(suffix=".json", delete=False, mode='w') as temp:
    json.dump(messages, temp)
    temp_file_name = temp.name

# Invoke the Phi-3 model and get the response
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name="aistudio-nbrady-phi-3",
    deployment_name="phi-3-mini-4k-instruct",
    request_file=temp_file_name,
)

# Parse the response and add it to the input data
response_json = json.loads(response)["output"]
response_dict = {'content': response_json, 'role': 'assistant'}
messages['input_data']['input_string'].append(response_dict)

# Print the updated input data
print(json.dumps(messages["input_data"]["input_string"],indent=2))

### Preparing the Sample Input
You still with me? Good! In this section, we're doing the same as before, but we are preparing the input data for the Phi-3 model continuing with the sample we created in the previous section.

First, we define the input data. This includes the user's message and some parameters for the model. The parameters control the randomness of the model's output. The user's message is the first element of the selected sample from the previous section.

In [ ]:
import json
import tempfile
import random

# Get a random sample from the examples
i = random.randint(0, len(examples) - 1)
sample = examples[i]

# Define the input data
messages = { "input_data": { 
    "input_string": [{"role": "user", "content": sample[0]}], 
    "parameters": { "temperature": 0.7, "top_p": 0.9, "do_sample": True, "max_new_tokens": 1000, }, }
}

# Write the input data to a temporary file
with tempfile.NamedTemporaryFile(suffix=".json", delete=False, mode="w") as temp:
    json.dump(messages, temp)
    temp_file_name = temp.name

# Invoke the Phi-3 model and get the response
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name="aistudio-nbrady-phi-3",
    deployment_name="phi-3-mini-4k-instruct",
    request_file=temp_file_name,
)

# Parse the response and add it to the input data
response_json = json.loads(response)["output"]
response_dict = {'content': response_json, 'role': 'assistant'}
messages['input_data']['input_string'].append(response_dict)

# Print the updated input data
print(json.dumps(messages["input_data"]["input_string"],indent=2))

### Building the Chat Interface
In this section, we are building a chat interface using Gradio, a Python library for creating UIs for machine learning models.

First, we define a function `predict` that takes a message and a history of previous messages as input. This function prepares the input data for the Phi-3 model, invokes the model, and processes the model's response.

```python
def predict(message, history):
    messages = {
        "input_data": {
            "input_string": [ ],
            "parameters": { "temperature": 0.6, "top_p": 0.9, "do_sample": True, "max_new_tokens": 1000, }, } }
    for user, assistant in history:
        messages["input_data"]["input_string"].append({"content": user, "role": "user"})
        messages["input_data"]["input_string"].append({"content": assistant, "role": "assistant"})
    messages["input_data"]["input_string"].append({"content": message, "role": "user"})
    ...
```
We will use the same `tempile`, `invoke`, and `response` logic as we had before.

After defining the `predict` function, we create a Gradio interface for it. This interface includes a textbox for the user to enter their message and a chatbot to display the conversation history. We also provide some example conversations to help users understand how to interact with the chatbot.

> We also provided some example conversations (`examples` parameter in `gr.ChatInterface`) to help users understand how to interact with the chatbot. These examples were generated from an optional dataset. However, if you don't have such a dataset or prefer not to use it, you can simply remove the `examples` parameter from the `gr.ChatInterface` call.

### Launch the Gradio interface
```python
gr.ChatInterface(
    fn=predict,
    textbox=gr.Textbox(
        value="I am going to Paris, what should I see?",
        placeholder="Ask me anything...",
        scale=5,
        lines=3,
    ),
    chatbot=gr.Chatbot(render_markdown=True),
    examples=examples,
    title="Phi-3: Tiny but mighty!",
    fill_height=True,
).launch()
```
This process creates a user-friendly chat interface for the Phi-3 model.

In [ ]:
import gradio as gr
import json
import tempfile


def predict(message, history):
    # Define the input data
    messages = {
        "input_data": {
            "input_string": [],
            "parameters": {
                "temperature": 0.6,
                "top_p": 0.9,
                "do_sample": True,
                "max_new_tokens": 1000,
            },
        }
    }
    for user, assistant in history:
        messages["input_data"]["input_string"].append({"content": user, "role": "user"})
        messages["input_data"]["input_string"].append({"content": assistant, "role": "assistant"})
    messages["input_data"]["input_string"].append({"content": message, "role": "user"})

    # Write the input data to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".json", delete=False, mode="w") as temp:
        json.dump(messages, temp)
        temp_file_name = temp.name

    # Invoke the Phi-3 model and get the response
    response = workspace_ml_client.online_endpoints.invoke(
        endpoint_name="aistudio-nbrady-phi-3",
        deployment_name="phi-3-mini-4k-instruct",
        request_file=temp_file_name,
    )

    # Parse the response and add it to the input data
    response_json = json.loads(response)["output"]
    response_dict = {"content": response_json, "role": "assistant"}
    messages["input_data"]["input_string"].append(response_dict)
    
    return response_json

# Launch the Gradio interface
gr.ChatInterface(
    fn=predict,
    textbox=gr.Textbox(
        value="I am going to Paris, what should I see?",
        placeholder="Ask me anything...",
        scale=5,
        lines=3,
    ),
    chatbot=gr.Chatbot(render_markdown=True, height=500),
    examples=examples, # This is the optional list of examples we created earlier
    title="Phi-3: Tiny but mighty!",
).launch()

### Conclusion

In this blog post, we've walked through the process of building a user-friendly chat interface for the Phi-3 model using Gradio. This interface includes a textbox for the user to enter their message and a chatbot to display the conversation history. 

By creating this interface, we've made the Phi-3 model more accessible and easier to demo, allowing users to interact with it in a natural, conversational manner. This is a great example of how small learning models can be integrated into applications using natural language.